In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import os

In [4]:
from supabase import create_client, Client

In [8]:
supabase_url = os.environ.get("SUPABASE_URL")

In [6]:
supabase_api_key = os.environ.get("SUPBASE_KEY")

In [7]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [24]:
def fetch_result(date):
    result = supabase.table('webhooks')\
        .select('id', 'news_output', 'source_urls', 'news_date', 'monitor_type')\
    .eq('news_date', date)\
        .order('created_at', desc=False)\
        .execute()

    result = result.data
    
    return result

In [25]:
fetch_result('2026-02-21')

[{'id': 1297,
  'news_output': 'The Guardian (citing the Financial Times) reports on large proposed investments into OpenAI (coverage of Nvidia and other potential investors) and notes earlier reporting of a larger $100bn figure that was later reduced in subsequent coverage — a major funding/ownership development in the AI sector.',
  'source_urls': ['https://www.theguardian.com/technology/2026/feb/20/nvidia-investment-openai-chatgpt-funding-round-ai-artificial-intelligence'],
  'news_date': '2026-02-21',
  'monitor_type': 'general'},
 {'id': 1298,
  'news_output': 'Coverage of the India AI Impact Summit reports large infrastructure-linked investment commitments (over $250 billion cited by Indian ministers) and an AI Impact Summit declaration signed by major countries — significant policy and investment-level commitments affecting AI deployment and international collaboration.',
  'source_urls': ['https://m.economictimes.com/news/newsblogs/ai-summit-2026-live-updates-day4-bharat-mandap

In [30]:
def unique_result(date):

    result = fetch_result(date)

    unique_results = []
    
    for i in result:
        current_news = i['news_output']
        if not any(item['news_output'] == current_news for item in unique_results):
            unique_results.append(i)
            
    return unique_results

In [31]:
unique_result('2026-02-21')

[{'id': 1297,
  'news_output': 'The Guardian (citing the Financial Times) reports on large proposed investments into OpenAI (coverage of Nvidia and other potential investors) and notes earlier reporting of a larger $100bn figure that was later reduced in subsequent coverage — a major funding/ownership development in the AI sector.',
  'source_urls': ['https://www.theguardian.com/technology/2026/feb/20/nvidia-investment-openai-chatgpt-funding-round-ai-artificial-intelligence'],
  'news_date': '2026-02-21',
  'monitor_type': 'general'},
 {'id': 1298,
  'news_output': 'Coverage of the India AI Impact Summit reports large infrastructure-linked investment commitments (over $250 billion cited by Indian ministers) and an AI Impact Summit declaration signed by major countries — significant policy and investment-level commitments affecting AI deployment and international collaboration.',
  'source_urls': ['https://m.economictimes.com/news/newsblogs/ai-summit-2026-live-updates-day4-bharat-mandap